<a href="https://colab.research.google.com/github/JacBGar/DSCI326/blob/main/Activity1_4_DSCI326.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(dplyr)
library(ggplot2)
library(readr)
library(clinfun)

df <- read_csv("no_sensitive_Secondary+Traumatic+Stress+and+its+Effects+on+Nurse+Faculty_February+1,+2022_16.37.csv")  # Replace with your file path

df <- df %>% mutate(Participant_Number = row_number())

q41_mapping <- c("Never" = 1, "Rarely" = 2, "Occasionally" = 3, "Often" = 4, "Very Often" = 5)
q39_mapping <- c("None" = 1, "A Little" = 2, "Some" = 3, "A Lot" = 4, "A Great Deal" = 5)

df <- df %>%
  mutate(across(starts_with("Q39_"), ~ as.numeric(q39_mapping[.]))) %>%
  mutate(across(starts_with("Q41_"), ~ as.numeric(q41_mapping[.])))

q41_columns <- paste0("Q41_", 1:17)
df <- df %>%
  rowwise() %>%
  mutate(STS = sum(c_across(all_of(q41_columns)), na.rm = TRUE)) %>%
  ungroup()

df <- df %>%
  mutate(STS_Category = case_when(
    STS <= 27 ~ "Little or No STS",
    STS >= 28 & STS <= 37 ~ "Mild STS",
    STS >= 38 & STS <= 43 ~ "Moderate STS",
    STS >= 44 & STS <= 48 ~ "High STS",
    STS >= 49 ~ "Severe STS"
  ))

q39_columns <- paste0("Q39_", 1:7)
df <- df %>%
  rowwise() %>%
  mutate(`Functional Impairment` = sum(c_across(all_of(q39_columns)), na.rm = TRUE)) %>%
  ungroup()

summary_table <- df %>%
  group_by(STS_Category) %>%
  summarise(Mean_Functional_Impairment = mean(`Functional Impairment`, na.rm = TRUE)) %>%
  arrange(factor(STS_Category, levels = c("Little or No STS", "Mild STS", "Moderate STS", "High STS", "Severe STS")))

print(summary_table)

ggplot(df, aes(x = factor(STS_Category, levels = c("Little or No STS", "Mild STS", "Moderate STS", "High STS", "Severe STS")), y = `Functional Impairment`)) +
  geom_boxplot(fill = "lightblue") +
  theme_minimal() +
  labs(title = "Functional Impairment Scores by STS Category",
       x = "STS Category",
       y = "Functional Impairment Score") +
  theme(plot.title = element_text(hjust = 0.5))

df <- df %>%
  mutate(STS_Category = factor(STS_Category,
                               levels = c("Little or No STS", "Mild STS",
                                          "Moderate STS", "High STS", "Severe STS"),
                               ordered = TRUE))

jt_test <- jonckheere.test(
  x = df$`Functional Impairment`,
  g = df$STS_Category,
  alternative = "increasing"
)

print(jt_test)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error in library(clinfun): there is no package called ‘clinfun’
